# Multivariate Logistic Regression
### Handwriting Recognition Data
##### From Exercise 3

In [5]:
#The limitation with our simple and regularized logistic regression is that is only works for binary classification (ex. Accepted or Rejected).
#This section will find the solution to handle multi-class classification and set ourselves up for neural networks in the next section.

#We will use logistic regression to recognize hand-written digits (0-9).
#Data is in a MATLAB format and need to use SciPy utility to load it.

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
%matplotlib inline

#os.chdir('..')
path = os.getcwd() + '\data\ex3data1.mat'
data = loadmat(path)
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [7]:
#Examine the shape of the arrays from the .mat file
data['X'].shape, data['y'].shape

#There are 5000, 400-dimensional vectors. The 400 features represent the grayscale intensities of each pixel in the 20x20 image
#The second 5000x1 matrix represents the actual digit in that image.

((5000, 400), (5000, 1))

In [9]:
#We need to vectorize our functions for logistic regression (no for loops). They will be far more efficient and they will be faster.
#We can do this by taking advantage of linear algebra optimization.
#The cost function was already vectorized, so it is the same below as before.

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def cost(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    reg = (learningRate / 2 * len(X)) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))
    return np.sum(first - second) / (len(X)) + reg

#To get rid of the for loops in the gradient function, we need to compute the gradient for each parameter at once using linear algebra

def gradient(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    parameters = int(theta.ravel().shape[1])
    error = sigmoid(X * theta.T) - y
    
    grad = ((X.T * error) / len(X)).T + ((learningRate / len(X)) * theta)
    
    # intercept gradient is not regularized
    grad[0, 0] = np.sum(np.multiply(error, X[:,0])) / len(X)
    
    return np.array(grad).ravel()

#Taking a step back and looking at what we are doing in a nutshell:
#The cost function says: "given some candidate solution theta applied to input X, how far off is the result from the true desired outcome y"
#This cost function is our method that evaluates a set of parameters.
#The gradient function specifies how to change those parameters to get an answer that is slightly better than the previous.


In [11]:
#Now we need to build a classifier. 
#We have 10 possible classes (0-9), so we need a strategy to deal with a multi-class scenario.
#We use a one-vs-all classification approach, where a label with k different classes results in k classifiers each deciding between "class i" and "not class i".
#Wrap this classifier up in a function that computes the final weights for each of the 10 classifies and returns the weights as a kX(n+1) array where n is the number of parameters.

from scipy.optimize import minimize

def one_vs_all(X, y, num_labels, learning_rate):
    rows = X.shape[0]
    params = X.shape[1]
    
    # k X (n + 1) array for the parameters of each of the k classifiers
    all_theta = np.zeros((num_labels, params + 1))
    
    # insert a column of ones at the beginning for the intercept term
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    
    # labels are 1-indexed instead of 0-indexed
    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (rows, 1))
        
        # minimize the objective function
        fmin = minimize(fun=cost, x0=theta, args=(X, y_i, learning_rate), method='TNC', jac=gradient)
        all_theta[i-1,:] = fmin.x
    
    return all_theta

In [12]:
#Check that training function actually runs and gets sensible values
all_theta = one_vs_all(data['X'], data['y'], 10, 1)
all_theta

array([[-3.70247930e-05,  0.00000000e+00,  0.00000000e+00, ...,
        -2.24803609e-10,  2.31962912e-11,  0.00000000e+00],
       [-8.96250707e-05,  0.00000000e+00,  0.00000000e+00, ...,
         7.26120806e-09, -6.19965281e-10,  0.00000000e+00],
       [-8.39553409e-05,  0.00000000e+00,  0.00000000e+00, ...,
        -7.61695659e-10,  4.64917668e-11,  0.00000000e+00],
       ...,
       [-7.00832281e-05,  0.00000000e+00,  0.00000000e+00, ...,
        -6.92008854e-10,  4.29241394e-11,  0.00000000e+00],
       [-7.65188037e-05,  0.00000000e+00,  0.00000000e+00, ...,
        -8.09503437e-10,  5.31058807e-11,  0.00000000e+00],
       [-6.63412525e-05,  0.00000000e+00,  0.00000000e+00, ...,
        -3.49765971e-09,  1.13668565e-10,  0.00000000e+00]])

In [13]:
#Now define a vectorized prediction function that outputs the class label as the class with highest probability
def predict_all(X, all_theta):
    rows = X.shape[0]
    params = X.shape[1]
    num_labels = all_theta.shape[0]
    
    # same as before, insert ones to match the shape
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    
    # convert to matrices
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)
    
    # compute the class probability for each class on each training instance
    h = sigmoid(X * all_theta.T)
    
    # create array of the index with the maximum probability
    h_argmax = np.argmax(h, axis=1)
    
    # because our array was zero-indexed we need to add one for the true label prediction
    h_argmax = h_argmax + 1
    
    return h_argmax

In [15]:
#Use the predict function to see how well our classifier works.
y_pred = predict_all(data['X'], all_theta)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print('accuracy = {0}%'.format(accuracy * 100))

accuracy = 74.6%
